
TODO:
* Function for remove a set of columns / rows and check theyve been removed by returning bool
* Can do PCA but mostly note for financial loan data as its very hard to be explainable. Can maybe save it for later
* Can do stratify Y
* Write a helper func to display the data dict
* verify drop hardship loans works
* pyment_plan variation is low, do vif. May also be leaky if its put inplace after. It is
* title vs purpose. Both are good. Drop one
* func that takes in some columns and returns the data dict entry for them
* check disbursement_method
* initial_list_status ie https://www.fintechnexus.com/lending-club-whole-loan-program-one-year-later/
* Drop issue_d after EDA
* Make the columns to drop into a list, so that i can keep track and seperate which oclumns im dropping and why
* funded_amnt', 'funded_amnt_inv
* - Loan-to-income ratio
* when loading in the data make it to select inplace
* Policy code 2
* ***Amount Loan was funded for***
We will drop the funded amount as it leaks that the loan has passed the application process.
#sample_accepted_df.drop(columns = ['funded_amnt', 'funded_amnt_inv'], inplace=True)
* pip list --format=freeze > requirements.txt

# Data Cleaning

## Date: OCT 10, 2023

-- ------------------------


## Introduction

This notebook cleans the data for the lending club accepted loans, then exports the data as a parquet file. Due to the size of the dataset, the csv is read in chunks, with a random sample taken from each each chunk. Only fully paid and charged off / defaulted loans are sampled as current loans hold no value in classifying the target variable. This allows us to more efficiently load the data. Those samples are merged and will become the working dataset for the duration of the project. After unnecessary and leaky features are removed, features are formatted and null values dealt with. Finally the dataframe is size is optimized and exported.  

### Table-of-contents


1. [Introduction](#Introduction)
   - [Table-of-contents](#Table-of-contents)
   - [Import-Librarys](#Import-Librarys)
   - [Data Dictionary](#Data-Dictionary)
   - [Define-Functions](#Define-Functions)
   - [Load in the data](#Load-the-data)
3. [Data Cleaning](#Data-Cleaning)
   - [Initial Exploration](#Initial-Exploration)
   - [Feature Pruning](#Feature-Pruning)
   - [Explore Columns to drop](#Explore-Columns-to-drop)
   - [Dataframe Null Values](#Dataframe-Null-Values)
4. [Dataframe optimization](#Dataframe-optimization)
5. [Exploratory-Data-Analysiss](Exploratory-Data-Analysis)
6. [Feature Engineering](#Feature-Engineering)
7. . [Conclusion](#Conclusion)


### Import-Librarys

In [ ]:
#%pip install pandas-downcast

In [ ]:
#%pip install missingno

In [ ]:
%pip install pyarrow

In [ ]:
#%run helpers.ipynb

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#from helpers import full_display
import pdcast as pdc
#import missingno as msno

from pathlib import Path

### Data-Dictionary

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#pathlib is used to ensure compatibility across operating systems
try:
    data_destination = Path('../Data/Lending_club/Lending Club Data Dictionary Approved.csv')
    dict_df = pd.read_csv(data_destination, encoding='ISO-8859-1')
    display(dict_df.iloc[:,0:2])
except FileNotFoundError as e:
    print(e.args[1])
    print('Check file location')

#### Define-Functions

In [ ]:
def map_emp_length(string):
    if string == '< 1 year':
        return 0.5
    elif string == '10+ years':
        return 10
    elif 'years' or 'year' in string:
        return int(string.split()[0])
    elif string == '0':
        return 0
    else:
        return string

When initially loading in the dataset, Pandas raised a DtypeWarning over mixed datatypes within various columns. Setting low_memory = False while breaking the CSV into chunks allows Pandas to load an entire chunk before guessing the data types. When the script to scrape the data dictionary is finished, the data dict can then be passed in instead of relying on pandas. The mixed_data_types function is stilled called as a sanity check.

In [ ]:
def mixed_data_types(df:pd.DataFrame) -> bool:
    '''
    Takes in a dataframe and checks for columns with mixed data types
    If none are found return False, else True
    
    :param df: The dataframe to be checked
    :type df: obj
    :return bool: True if found, false if none were found
    :type return: bool
    '''
    
    #loop through each column
    for column in df:

        #filter outint datatypes coming from Nan and get unique data types
        unique_types = df[column].dropna(inplace=False).apply(type).unique()

        #if there are more than 1 datatype in a column
        if unique_types.size > 1:
            return True
    return False

#### Load the data

Due to the size of the dataset, it is read in chunks. After each chunk is read and checked for mixed data types, it is randomly sampled and then placed within a list. Only fully paid and /defaulted and charged off loans are taken, as current loans including late or in grace period loans do not hold any value in target variable prediction. This is done when loading in the data otherwise it becomes too large for memory. The different samples are then combined into a single sample representative of the whole dataset. EDA will be performed on this single sample.

In [ ]:
chunk_size = 5*100000
sample_size =  100000
random_state = 11

assert sample_size < chunk_size, f"Cannot take a sample of {sample_size} rows out of {chunk_size} rows"

print(f'Chunk size: {chunk_size} rows')
print(f'Rows to be sampled: {sample_size} rows')


sampled_dataframes = []
try:
    data_destination = Path('../Data/Lending_club/accepted_2007_to_2018Q4.csv')

    #split the csv into chunks and iterate over each chunk
    with pd.read_csv(data_destination, chunksize=chunk_size, low_memory = False) as reader:
        for count,chunk in enumerate(reader):
            
            if mixed_data_types(df=chunk) == True:
                raise Exception("Mixed data types found")

            #define a list that includes only finished loan statuses
            finished_loan_status = ['Fully Paid',
                                    'Charged Off',
                                    'Does not meet the credit policy. Status:Fully Paid',
                                    'Does not meet the credit policy. Status:Charged Off',
                                    'Default']
                        
            #filter the dataframe for loans that are finished or null
            filtered_chunk = chunk.loc[chunk['loan_status'].isin(finished_loan_status) | chunk['loan_status'].isnull()]
            
            #sample the filtered df and append to list
            sampled_df = filtered_chunk.sample(n=sample_size, random_state=random_state)
            sampled_dataframes.append(sampled_df)
            
            print(f"{count} sampled dataframe shape: {sampled_df.shape}")
        print('Finished')

except FileNotFoundError as e:
    print(e.args[1])
    print('Check file name and location')
    
except Exception as e:
    print(e.args[1])

There are no duplicate datatypes within any columns. The random samples can be combined into a single sample dataframe. This sample will be used as the working dataset.

In [ ]:
sample_accepted_df = pd.concat(sampled_dataframes, ignore_index=False)

&nbsp;

## Data Cleaning

### Initial Exploration

***Display the first 5 rows*** 

In [ ]:
sample_accepted_df.head(5)

***Dataframe shape***

In [ ]:
rows, columns = sample_accepted_df.shape
print(f'Dataframe rows: {rows}')
print(f'Dataframe columns: {columns}')

***Dataframe info***

In [ ]:
sample_accepted_df.info()

Of the 151 columns, 113 are float64 and 38 are objects. The dataframe takes up approximatly 580 MB.
Note:
- The numeric columns are all float64 and the object columns. These columns can be optimized later to save memory space and decrease computation time by changing the datatypes.
- There is no datetime column

***Describe Dataframe***

In [ ]:
sample_accepted_df.describe()

Some key points:

- Loan Amount
  
    - Average Loan Amount is ~ 15,000 USD with a standard deviation of 9240 USD, having a max of 40,000 USD and minimum of 500 USD. This follows LendingClubs  policies for minimum and maximum loan amounts.

- Funded amount
    - Nearly identical to the loan amount

- Funded amount by investors
    - Very similar to the  funded amount

- Interest Rate
    - The interest rates are quite high. An average of 13%, with a minimum of 5.3% and a maximum of 31%.


   

***Null Values***

Some rows are fully NaN values, aside from the id. This will cause issues when we try to inspect each column later. So we will drop `id` and the NaN rows, along with any other irrelevant columns including:  
- member_id
- url for the loan
- LC policy code
- title (information is already found under purpose)

In [ ]:
sample_accepted_df.drop(columns=['id', 'member_id', 'url', 'policy_code', 'title'], inplace=True)

In [ ]:
null_rows = sample_accepted_df.isnull().all(axis=1).sum()
print(f"Number of Null rows: {null_rows}")

In [ ]:
#drop rows that are all Nan
sample_accepted_df.dropna(how='all', inplace=True)

In [ ]:
null_rows = sample_accepted_df.isnull().all(axis=1).sum()
print(f"Number of Null rows: {null_rows}")

&nbsp;

---------------------------------------------

### Feature Pruning

We will exclude any leaky features, non relevant features and any features that were not present in the original loan application.

#### ***Irrelevant columns***

***Secondary Applicants Information***

The columns for the secondary applicant are largely nulls, so we will drop them

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

In [ ]:
sample_accepted_df['application_type'].value_counts()

In [ ]:
# Get the index of the loans where revol_bal_joint is not null
rows_to_remove = sample_accepted_df.loc[sample_accepted_df['application_type'] == 'Joint App'].index

# Drop the loans
sample_accepted_df.drop(rows_to_remove, inplace=True)

sample_accepted_df.drop(columns = ['revol_bal_joint', 'sec_app_fico_range_low', 
                                   'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
                                   'sec_app_inq_last_6mths', 'sec_app_mort_acc',
                                   'sec_app_open_acc', 'sec_app_revol_util', 
                                   'sec_app_open_act_il', 'sec_app_num_rev_accts', 
                                   'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med',
                                   'sec_app_mths_since_last_major_derog',
                                   'verification_status_joint', 'dti_joint',
                                   'annual_inc_joint', 'application_type'], inplace=True)

***Loan Grade***

We will also drop the credit rating assigned by Lendingclub as we want to do our own assessment

In [ ]:
sample_accepted_df.drop(columns=['grade','sub_grade'], inplace=True)

***Hardship Loans***

Hardship loans make up a very small proportion of the dataset, and add 15 columns of complexity. We will drop these columns and loans if they exist in our dataset, and limit our analysis to non hardship loans.

In [ ]:
#fetch the value counts for the for the hardships flags
hardships = sample_accepted_df['hardship_flag'].value_counts()
display(hardships)

#if there are loans with the yes hardship flag
if 'Y' in hardships:
    #get the count of hardship loans
    yes_hardship_count = hardships.iloc[1]
    print(f'The hardship loans represent only {(yes_hardship_count/sample_accepted_df.shape[0])*100}% of the dataset')

    #get the index of the hardship loans
    rows_to_remove = sample_accepted_df.loc[sample_accepted_df['hardship_flag'] == 'Y'].index

    #drop the loans
    sample_accepted_df.drop(rows_to_remove, inplace=True)

    #check the rows have been dropped
    assert sample_accepted_df['hardship_flag'].value_counts().shape[0] == 1
    print('Hardship loans and associated columns have been dropped')

else:
    print('There are no hardship loans.')
    
columns_to_drop = ['hardship_flag', 'hardship_type',
                        'hardship_reason', 'hardship_status',
                        'hardship_amount', 'hardship_start_date',
                        'hardship_end_date', 'deferral_term',
                        'hardship_length', 'hardship_dpd',
                        'hardship_loan_status', 'payment_plan_start_date',
                        'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
                        'hardship_last_payment_amount']

sample_accepted_df.drop(columns = columns_to_drop, inplace=True)
print('Hardship columns have been dropped')

***Employee Title***

In [ ]:
sample_accepted_df['emp_title'].value_counts()

In [ ]:
print(sample_accepted_df.shape)

There are too many unique Employee titles to attempt any sort of grouping or encoding for now. Possibly in the future we could use NLP or an external API to group the Employee Title.

In [ ]:
sample_accepted_df.drop(columns = 'emp_title', inplace=True)

***Loan Status***

Any current loans have already been dropped when reading in the data. We can now finish grouping the completed loans.

More information on the loan status's can be found here:  
https://www.lendingclub.com/help/investing-faq/what-do-the-different-note-statuses-mean  
https://www.fintechnexus.com/policy-code-2-loans-lending-club/

In [ ]:
sample_accepted_df['loan_status'].value_counts()

The "Does not meet the credit policy" means when the loans were made under a different credit card policy, that does not meet the current policy. This has affect on the loans themselves, so they can be grouped with their counter parts. Charged off and Defaulted can also been grouped together.

In [ ]:
status_mapping = {
    "Fully Paid": "Fully Paid",
    "Does not meet the credit policy. Status:Fully Paid": "Fully Paid",
    "Does not meet the credit policy. Status:Charged Off": "Charged Off/Default",
    "Charged Off": "Charged Off/Default",
    "Default": "Charged Off/Default",
}

#map the loans
sample_accepted_df['loan_status'] = sample_accepted_df['loan_status'].map(status_mapping)

Check the mapping has worked:

In [ ]:
sample_accepted_df['loan_status'].value_counts()

The mapping was successful, we are not left with only successful and failed loans.

***State / Zip Code***

We have 2 geographical features. We will drop both of them for now as they will add too much complexity to the model. However, in the future we can perhaps use a 3rd party api and introduce mean or median income data by region, allowing us to capture some of that geographical data.

In [ ]:
display(sample_accepted_df['addr_state'].value_counts())
print('-'*20)
display(sample_accepted_df['zip_code'].value_counts())

In [ ]:
sample_accepted_df.drop(columns = ['zip_code', 'addr_state'], inplace=True)

***Fico scores***

We can drop the fico scores

In [ ]:
sample_accepted_df.drop(columns = ['last_fico_range_high', 'last_fico_range_low',
                                   'fico_range_low', 'fico_range_high'], inplace=True)

***Description***

In [ ]:
display(sample_accepted_df['desc'].value_counts())

There are too many unique descriptions to create dummy variables. We can drop this column

In [ ]:
sample_accepted_df.drop(columns = ['desc'], inplace=True)

#### Leaky columns

We will remove any columns that can leak the outcome of the application ie, any data the originates after a loan has been funded or rejected.

In [ ]:
sample_accepted_df.info()

We can remove any columns that:  
- describe payments made toward the loan

In [ ]:
sample_accepted_df.drop(columns = ['total_pymnt', 'total_rec_prncp',
                       'total_rec_int', 'total_rec_late_fee',
                       'recoveries', 'collection_recovery_fee',
                       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'total_pymnt_inv', 
                                   'collections_12_mths_ex_med', 'collection_recovery_fee',
                                  'mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog', 'initial_list_status'], inplace=True)

- loan attributes post acceptance

In [ ]:
sample_accepted_df.drop(columns=['out_prncp', 'out_prncp_inv',
                                 'pymnt_plan',
                                'acc_now_delinq', 'last_credit_pull_d',
                                'debt_settlement_flag_date', 'settlement_term',
                                 'total_il_high_credit_limit', 'total_bc_limit',
                                 'num_tl_120dpd_2m', 'num_tl_30dpd', 'disbursement_method',
                                ], inplace=True)

- any settlement information

In [ ]:
sample_accepted_df.drop(columns=['debt_settlement_flag', 'settlement_status',
                                 'settlement_date', 'settlement_amount',
                                 'settlement_percentage'], inplace=True)

In [ ]:
sample_accepted_df.head(5)

### Feature engineering

***Term***

Convert from str to int

In [ ]:
sample_accepted_df['term'].value_counts()

In [ ]:
# Remove the "months" text and convert to int
sample_accepted_df['term'] = sample_accepted_df['term'].str.extract('(\d+)').astype('int8')

In [ ]:
sample_accepted_df['term'].value_counts()

In [ ]:
sample_accepted_df.head()

***Emp_Length***

We will map greater than 10 years to 10, less than 1 year to 0.5 as to differentiate it between 1 and 0, preserving that information.

In [ ]:
sample_accepted_df['emp_length'].value_counts()

In [ ]:
sample_accepted_df['emp_length'].fillna(value='0',inplace=True)

In [ ]:
sample_accepted_df['emp_length'] = sample_accepted_df['emp_length'].apply(map_emp_length)

In [ ]:
sample_accepted_df['emp_length'].value_counts()

### Dataframe-Null-Values

------------------------------------------

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
sample_accepted_df.isnull().sum().sort_values(ascending=False)

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

Note how there seems to be groupings of nulls. We will explore these groupings

&nbsp;

***Explore the groupings of nulls***

***Debt to Income***

In [ ]:
sample_accepted_df['dti'].value_counts()

In [ ]:
null_rows = sample_accepted_df[sample_accepted_df['dti'].isnull()]
print('Number of null rows in dti: ', sample_accepted_df['dti'].isnull().sum())
display(null_rows)

We can see that if there is no reported income, then dti is NaN as you can't divide by 0. This is a key metric used in evaluating a borrowers creditworthiness. We will fill it with a large dummy variable (10000) to preserve some of the information the ratio is trying to convey; in this case if a borrower does not have any income. We could also drop these rows as well as there are only a few

In [ ]:
sample_accepted_df['dti'].fillna(value=10000,inplace=True)

Check the column has been updated

In [ ]:
sample_accepted_df['dti'].isnull().sum()

***inq_last_12m***

In [ ]:
sample_accepted_df['inq_last_12m'].isnull().sum()

***open_acc_6m***

We can see from the nulls values that there is a grouping of nulls that make up approximatly 0.48% of the dataset. We will explore these

In [ ]:
sample_accepted_df['open_acc_6m'].isnull().sum()

https://www.fintechnexus.com/lending-club-adds-15-new-fields-and-folio-introduces-a-true-secondary-market-api/

In [ ]:
sample_accepted_df.drop(columns= ['max_bal_bc','open_rv_24m',
                                  'open_rv_12m','inq_fi',
                                  'total_bal_il','inq_last_12m',
                                  'open_il_24m','open_il_12m',
                                  'open_act_il','total_cu_tl','open_acc_6m'], inplace=True)

In [ ]:
sample_accepted_df.head()

In [ ]:
sample_accepted_df.info()

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

In [ ]:
columns_to_drop = ['mths_since_recent_bc_dlq','mths_since_recent_revol_delinq',
                   'il_util','mths_since_recent_inq','mths_since_rcnt_il',
                   'mo_sin_old_il_acct','bc_util',
                   'bc_open_to_buy','percent_bc_gt_75',
                   'mths_since_recent_bc','all_util','revol_util']


In [ ]:
sample_accepted_df.drop(columns = columns_to_drop, inplace=True)

In [ ]:
sample_accepted_df.shape

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

In [ ]:
sample_accepted_df.dropna(subset=['annual_inc', 'total_acc', 
                                  'tax_liens', 'chargeoff_within_12_mths', 
                                  'pub_rec_bankruptcies', 'total_bal_ex_mort',
                                  'tot_hi_cred_lim', 'avg_cur_bal', 'pct_tl_nvr_dlq'
                                 ], inplace=True)

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

### Dataframe Optimization

In [ ]:
#print(sample_accepted_df.info())
#sample_accepted_df = pdc.downcast(sample_accepted_df)
#print(sample_accepted_df.info())
# Infer minimum schema for DataFrame.
#schema = pdc.infer_schema(sample_accepted_df)
#print(schema)
#sample_accepted_df.shape

TODO: Optimize column datatypes to reduce code runtime and increase memory efficiency

### Export Dataframe

In [ ]:
export_destination = Path('../Data/Lending_club/Cleaned')
sample_accepted_df.to_parquet(export_destination)

### Feature-Engineering

In [ ]:
['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'purpose',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'application_type']

## Conclusion

### Resources used:

- https://stackoverflow.com/questions/51325601/how-to-stop-my-pandas-data-table-from-being-truncated-when-printed